<hr style="height:2px;">

# Demo: Apply trained CARE model for image transfer in *BCM3D 2.0*

This notebook demonstrates applying a CARE model for a 3D image transfer task, assuming that training was already completed via training.ipynb.  
The trained model is assumed to be located in the folder `models` with the name `my_model`.

In [ ]:
from __future__ import print_function, unicode_literals, absolute_import, division
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

from tifffile import imread
from csbdeep.utils import Path, download_and_extract_zip_file, plot_some
from csbdeep.io import save_tiff_imagej_compatible
from csbdeep.models import CARE

import os

<hr style="height:2px;">

# Load model and input images


In [2]:
# Load model
model = CARE(config=None, name='my_model', basedir='models')
axes ='ZCYX'
#input images

x_images = []
x_filenames = []
folder = 'Data/Predict/Raw' #locate to your raw data
for filename in os.listdir(folder): #import input folder
    if filename.endswith('.tif'):
        img = imread(os.path.join(folder, filename))
        if img is not None:
            x_images.append(img)
            x_filenames.append(filename)

Loading network weights from 'weights_best.h5'.


## Apply CARE network to input image

Predict the restored image, we suggest you use n_tiles to divide your data if it is too large for memory.

In [ ]:
# Predict
img_num = len(x_images)
for k in range(img_num):
    x = x_images[k]
    x = np.stack((x, x), axis=1)
    print('input image size =', x.shape)
    print('input image axes =', axes)

    #%%time
    restored = model.predict(x, axes, n_tiles=(1,1,2,2)) #solve out of memory issue
    print('output image size=', restored.shape)
    print('output image axes =', axes)
    #Save Stored images
    Path('Data/Predict/ModelOutput').mkdir(exist_ok=True)
    save_tiff_imagej_compatible('Data/Predict/ModelOutput/%s_%s' % (model.name, x_filenames[k]), restored, axes)

**Note**: *Out of memory* problems during `model.predict` can also indicate that the GPU is used by another process. In particular, shut down the training notebook before running the prediction (you may need to restart this notebook).